In [6]:
from qdrant_client import QdrantClient
from qdrant_client import models
import json
import uuid

In [7]:
qdClient = QdrantClient("http://localhost:6333")

In [9]:
collection_name="stardew-sparse-and-dense"
vector_model_handle = "jinaai/jina-embeddings-v2-small-en"
EMBEDDING_DIMENSIONALITY = 512
spasrse_model_handle="Qdrant/bm25"

In [ ]:
qdClient.get_collection(collection_name)

In [ ]:
info = qdClient.get_collection(collection_name)
print(info.points_count, info.indexed_vectors_count)

In [ ]:
tables = []
texts = []

with open('data/summarized_tables.json', 'r' , encoding="utf-8") as f1:
    tables = json.load(f1)
    
for t in tables:
    t["content_type"] = "table"

with open('data/summarized_texts.json', 'r' , encoding="utf-8") as f2:
    texts = json.load(f2)

for t in texts:
    t["content_type"] = "text"

In [ ]:
qdClient.create_collection(
    collection_name=collection_name,
    vectors_config={
        # Named dense vector for jinaai/jina-embeddings-v2-small-en
        "jina-small": models.VectorParams(
            size=EMBEDDING_DIMENSIONALITY,
            distance=models.Distance.COSINE,
        ),
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

In [ ]:
points = []
for text in texts:
    text_to_embedd = f"Page title (2X importance): {text['page_title']}. Section title: {text['section_title']}. text: {text['text']}"
    
    point = models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=text_to_embedd,
                    model=vector_model_handle,
                ),
                "bm25": models.Document(
                    text=text_to_embedd, 
                    model=spasrse_model_handle,
                ),
            },
            payload=text
        )
    points.append(point)

In [ ]:
for table in tables:
    text_to_embedd = f"Page title (2X importance): {table['page_title']}. Section title: {table['section_title']}. Table summary: {table['summary']}"

    point = models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=text_to_embedd,
                    model=vector_model_handle,
                ),
                "bm25": models.Document(
                    text=text_to_embedd, 
                    model=spasrse_model_handle,
                ),
            },
            payload=table
        )
    points.append(point)


In [ ]:
def batch_upsert(qdClient, collection_name, points, batch_size=500):
    total = len(points)
    for i in range(0, total, batch_size):
        batch = points[i:i+batch_size]
        qdClient.upsert(collection_name=collection_name,points=batch)
        print(f"✅ Upserted {min(i+batch_size, total)}/{total}")

In [ ]:
batch_upsert(qdClient=qdClient,collection_name=collection_name,points=points,batch_size=1000)

In [ ]:
def multi_stage_search(client, collection_name, query: str, limit: int = 5) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model=vector_model_handle,
                ),
                using="jina-small",
                # Prefetch three times more results, then
                # expected to return, so we can really rerank
                limit=(3 * limit),
            ),
        ],
        query=models.Document(
            text=query,
            model=spasrse_model_handle, 
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [ ]:
question = "How to become friends with Abigail?"

results = multi_stage_search(client = qdClient , collection_name=collection_name, query=question)
results

In [ ]:
def rrf_search(client, collection_name, query: str, limit: int = 5) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model=vector_model_handle,
                ),
                using="jina-small",
                limit=(5 * limit),
            ),
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model=spasrse_model_handle,
                ),
                using="bm25",
                limit=(5 * limit),
            ),
        ],
        # Fusion query enables fusion on the prefetched results
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        with_payload=True,
    )

    return results.points[:limit]

In [ ]:
results = rrf_search(client=qdClient,collection_name=collection_name, query=question)
results

In [ ]:
results[0].payload

In [ ]:
def build_prompt(query, search_results):
    prompt_template = """
    You're an AI assistant for the players of a computer game named Stardew Valley.
    Answer the QUESTION based on the CONTEXT extracted from the game's wiki website. Some materials are texts and some are html tables.
    Use only the materials from the CONTEXT when answering the QUESTION, and don't use your own knowledge of the game.

    QUESTION: {question}

    CONTEXT:
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        if (doc.payload["content_type"] =="text"):
            context += (
            f"Page title: {doc.payload['page_title']}\n"
            f"Section title: {doc.payload['section_title']}\n"
            f"Text: {doc.payload['text']}\n\n\n"
            )
        elif (doc.payload["content_type"] =="table"):
            context += (
            f"Page title: {doc.payload['page_title']}\n"
            f"Section title: {doc.payload['section_title']}\n"
            f"Table HTML: {doc.payload['table_html']}\n\n\n"
            )

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [ ]:
prompt = build_prompt(question, results)

In [1]:
from Retrieval_evaluation import evaluate_search_functions

C:\Users\erfan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ragv1 import multi_stage_search , rrf_search 


In [3]:
evaluate_search_functions([multi_stage_search, rrf_search], k=5, sampleNum=5)

SUCCESS: Loaded 19039 text entries and 12010 table entries

Evaluating: multi_stage_search
multi_stage_search → MRR@5: 0.507, HitRate@5: 0.800

Evaluating: rrf_search
rrf_search → MRR@5: 0.300, HitRate@5: 0.400


{'multi_stage_search': {'MRR': 0.5066666666666667, 'HitRate': 0.8},
 'rrf_search': {'MRR': 0.3, 'HitRate': 0.4}}